<a href="https://colab.research.google.com/github/IanChoi2464/Machine-Learning-Colab/blob/main/9-3%20LSTM%20and%20GRU%20cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

순환 신경망에서 빼놓을 수 없는 핵심 기술인 LSTM과 GRU 셀을 사용한 모델을 만들어 본다.
# Keyword
**LSTM**: Long Short-Term Memory, LSTM 셀은 타임스텝이 긴 데이터를 효과적으로 학습하기 위해 고안된 순환층, 입력 게이트, 삭제 게이트, 출력 게이트 역할을 하는 작은 셀이 포함되어 있음\
**셀 상태**: LSTM 셀은 은닉 상태 외에 셀 상태를 출력, 셀 상태는 다음 층으로 전달되지 않으며 현재 셀에서만 순환됨\
**GRU**: Gated Recurrent Unit, GRU 셀은 LSTM 셀의 간소화 버전으로 생각할 수 있지만 LSTM 셀에 못지않는 성능을 냄
# Function
### **Tensorflow**
**LSTM**: LSTM 셀을 사용한 순환층 클래스\
첫 번째 매개변수에 뉴런의 개수를 지정\
dropout 매개변수에서 입력에 대한 드롭아웃 비율을 지정할 수 있음\
return_sequences 매개변수에서 모든 타임스텝의 은닉 상태를 출력할지 결정, 기본값은 False\
**GRU**: GRU 셀을 사용한 순환층 클래스\
첫 번째 매개변수에 뉴런의 개수를 지정\
dropout 매개변수에서 입력에 대한 드롭아웃 비율을 지정할 수 있음\
return_sequences 매개변수에서 모든 타임스텝의 은닉 상태를 출력할지 결정, 기본값은 False

In [ ]:
"""IMDB 데이터 준비하기"""
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=500)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

from tensorflow.keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input, maxlen=100)
val_seq = pad_sequences(val_input, maxlen=100)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
"""LSTM1"""
from tensorflow import keras

model = keras.Sequential()
model.add(keras.layers.Embedding(500, 16, input_length=100))
model.add(keras.layers.LSTM(8))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           8000      
                                                                 
 lstm (LSTM)                 (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 8809 (34.41 KB)
Trainable params: 8809 (34.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
"""LSTM2. 훈련함수"""
def model_fn(a_layer=None):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  model.add(keras.layers.Dense(100, activation='relu'))
  if a_layer:
    model.add(a_layer)
  model.add(keras.layers.Dense(10, activation='softmax'))
  return model

def train_fn(model, name):
  rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
  model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
  checkpoint_cb = keras.callbacks.ModelCheckpoint(name, save_best_only=True)
  early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
  history = model.fit(train_seq, train_target, epochs=100, batch_size=64, validation_data=(val_seq, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
"""LSTM2"""
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-lstm-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(train_seq, train_target, epochs=100, batch_size=64, validation_data=(val_seq, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
313/313 [==============================] - 20s 52ms/step - loss: 0.6925 - accuracy: 0.5360 - val_loss: 0.6917 - val_accuracy: 0.5734
Epoch 2/100
  3/313 [..............................] - ETA: 12s - loss: 0.6911 - accuracy: 0.5781

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 16s 52ms/step - loss: 0.6904 - accuracy: 0.5935 - val_loss: 0.6889 - val_accuracy: 0.6112
Epoch 3/100
313/313 [==============================] - 27s 85ms/step - loss: 0.6861 - accuracy: 0.6227 - val_loss: 0.6827 - val_accuracy: 0.6380
Epoch 4/100
313/313 [==============================] - 28s 89ms/step - loss: 0.6753 - accuracy: 0.6552 - val_loss: 0.6646 - val_accuracy: 0.6750
Epoch 5/100
313/313 [==============================] - 26s 82ms/step - loss: 0.6334 - accuracy: 0.7017 - val_loss: 0.5917 - val_accuracy: 0.7188
Epoch 6/100
313/313 [==============================] - 24s 75ms/step - loss: 0.5731 - accuracy: 0.7268 - val_loss: 0.5591 - val_accuracy: 0.7404
Epoch 7/100
313/313 [==============================] - 26s 83ms/step - loss: 0.5474 - accuracy: 0.7473 - val_loss: 0.5401 - val_accuracy: 0.7528
Epoch 8/100
313/313 [==============================] - 26s 83ms/step - loss: 0.5270 - accuracy: 0.7616 - val_loss: 0.5227 - val_accuracy: 0.76

In [ ]:
"""LSTM3"""
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
"""순환층에 드롭아웃"""
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(500, 16, input_length=100))
model2.add(keras.layers.LSTM(8, dropout=0.3))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model2.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-dropout-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
history = model2.fit(train_seq, train_target, epochs=100, batch_size=64, validation_data=(val_seq, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
"""2개의 층 연결하기1."""
model3 = keras.Sequential()
model3.add(keras.layers.Embedding(500, 16, input_length=100))
model3.add(keras.layers.LSTM(8, dropout=0.3, return_sequences=True))
model3.add(keras.layers.LSTM(8, dropout=0.3))
model3.add(keras.layers.Dense(1, activation='sigmoid'))

model3.summary()

In [ ]:
"""2개의 층 연결하기2."""
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model3.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-2rnn-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
history = model3.fit(train_seq, train_target, epochs=100, batch_size=64, validation_data=(val_seq, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
"""2개의 층 연결하기3."""
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
"""GRU1"""
model4 = keras.Sequential()
model4.add(keras.layers.Embedding(500, 16, input_length=100))
model4.add(keras.layers.GRU(8))
model4.add(keras.layers.Dense(1, activation='sigmoid'))

model4.summary()

rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model4.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-gru-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
history = model4.fit(train_seq, train_target, epochs=100, batch_size=64, validation_data=(val_seq, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
"""테스트"""
test_seq = pad_sequences(test_input, maxlen=100)
rnn_model = keras.models.load_model('best-2rnn-model.h5')
rnn_model.evaluate(test_seq, test_target)